## NEB

In [ ]:
from ase.optimize import BFGS
from ccs_fit.ase_calculator.ccs_ase_calculator import CCS
from tqdm import tqdm
from ase import io
import json

struct = io.read('DFT.db@32')
with open("CCS_params.json", 'r') as f:
    CCS_params = json.load(f)

CCS_calc = CCS(CCS_params=CCS_params)

#struct.rattle(0.05)
struct.calc = CCS_calc

opt = BFGS(struct, trajectory='opt_init.traj')
opt.run(fmax=0.005)

      Step     Time          Energy         fmax
BFGS:    0 15:37:31    -2389.008079       50.1746
BFGS:    1 15:41:15    -2483.927632       46.2723
BFGS:    2 15:45:04    -2536.291411       35.1515
BFGS:    3 15:48:11    -2577.236469       29.3873
BFGS:    4 15:51:11    -2617.343762       23.8680
BFGS:    5 15:54:05    -2684.704830       20.2190
BFGS:    6 15:56:42    -2737.422169       20.5801
BFGS:    7 15:59:15    -2870.318389       18.1747
BFGS:    8 16:01:40    -2880.035951       16.3698
BFGS:    9 16:04:09    -2947.793533       15.7568
BFGS:   10 16:06:34    -2941.811540       16.0746
BFGS:   11 16:09:01    -3032.171484       15.7759
BFGS:   12 16:11:29    -3068.154636       19.7522
BFGS:   13 16:38:15    -3202.487136       25.6927
BFGS:   14 16:43:29    -3294.013938       31.1917
BFGS:   15 16:46:44    -3454.758218       29.7147
BFGS:   16 16:49:55    -3552.777550       36.7745
BFGS:   17 16:53:04    -3719.986405       42.9145
BFGS:   18 16:55:52    -3819.303706       46.0493
B

In [21]:
from ase.visualize import view

struct = io.read('DFT.db@-1')

print(struct)
print(struct.get_potential_energy())

view(struct)

Atoms(symbols='Li60Ni104O216Ta4', pbc=True, cell=[[17.01695551, 0.0, 0.0], [-0.05602946, 14.44880072, 0.0], [-8.49742597, -0.04665949, 14.72109736]], calculator=SinglePointCalculator(...))
-2387.593537


<Popen: returncode: None args: ['C:\\Users\\Fremo211\\Anaconda3\\envs\\Tanta...>

In [ ]:
import ase

!ase gui opt_init.traj

In [4]:
from ase.visualize import view
init = io.read("opt_init.traj")

view(init)

<Popen: returncode: None args: ['C:\\Users\\Fremo211\\Anaconda3\\envs\\Tanta...>

In [7]:
final = init.copy()

final_pos = final.get_positions()
final_pos[24,:] = [-0.1, 3.2, 11.9]

final.set_positions(final_pos)

# view(final)

final.calc = CCS_calc
opt = BFGS(final, trajectory='final_opt.traj')
opt.run(fmax=0.005)

      Step     Time          Energy         fmax
BFGS:    0 13:52:54     -561.357678        0.0317
BFGS:    1 13:52:55     -561.357798        0.0315
BFGS:    2 13:52:56     -561.368203        0.0199
BFGS:    3 13:52:58     -561.368764        0.0282
BFGS:    4 13:52:59     -561.368854        0.0302
BFGS:    5 13:53:01     -561.369026        0.0324
BFGS:    6 13:53:02     -561.369442        0.0349
BFGS:    7 13:53:03     -561.370316        0.0357
BFGS:    8 13:53:05     -561.371728        0.0315
BFGS:    9 13:53:06     -561.372986        0.0209
BFGS:   10 13:53:07     -561.373508        0.0138
BFGS:   11 13:53:09     -561.373654        0.0149
BFGS:   12 13:53:10     -561.373698        0.0146
BFGS:   13 13:53:11     -561.373834        0.0132
BFGS:   14 13:53:13     -561.374085        0.0111
BFGS:   15 13:53:14     -561.374688        0.0101
BFGS:   16 13:53:16     -561.375679        0.0128
BFGS:   17 13:53:17     -561.376472        0.0235
BFGS:   18 13:53:18     -561.376700        0.0223
B

True

In [1]:
!ase gui final_opt.traj

'ase' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
from ase import io
from ase.neb import NEB
from ase.optimize import MDMin

# Read initial and final states:
initial = io.read('opt_init.traj')
final = io.read('final_opt.traj')

# Make a band consisting of 5 images:
images = [init]
images += [init.copy() for i in range(3)]
images += [final]
neb = NEB(images)

# Interpolate linearly the potisions of the three middle images:
neb.interpolate()

# Set calculators:
for image in images[1:4]:
    CCS_calc = CCS(CCS_params=CCS_params)
    image.calc = CCS_calc
    
# Optimize:
optimizer = MDMin(neb, trajectory='NEB.traj')
optimizer.run(fmax=0.05)

       Step     Time          Energy         fmax
MDMin:    0 16:24:00     -561.338510        0.1147
MDMin:    1 16:24:07     -561.339001        0.1116
MDMin:    2 16:24:13     -561.340382        0.1027
MDMin:    3 16:24:19     -561.342409        0.0893
MDMin:    4 16:24:25     -561.344762        0.0731
MDMin:    5 16:24:31     -561.347149        0.0568
MDMin:    6 16:24:37     -561.349372        0.0482


True

In [18]:
saddle_point = io.read('NEB.traj@-3')

view(saddle_point)

io.write("saddle.POSCAR", saddle_point)

In [17]:
neb_images = io.read('NEB.traj@-5:-1')
print(len(neb_images))

4


In [23]:
for i in range(3):
    view(neb_images[i+1])
    io.write(str(i+1) + ".POSCAR", neb_images[i+1])